In [27]:
import requests
import pdfplumber
import pandas as pd
import re
from collections import namedtuple

In [28]:
#Data Structure used to yield the results and will be converted to dataframe later
Data = namedtuple('Data', 'Date Ship_mode Balance_due Subtotal Discount Shipping Grand_total Order_ID' )
line_items = []

In [29]:
#How to extract the text from the PDF And view it in the notebook
pdf_path = 'invoice_Aaron Bergman_36258.pdf'

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    print(text)

SuperStore INVOICE
# 36258
Date: Mar 06 2012
Bill To: Ship To:
Ship Mode: First Class
Aaron Bergman 98103, Seattle,
Washington, United
Balance Due: $50.10
States
Item Quantity Rate Amount
Global Push Button Manager's Chair, Indigo 1 $48.71 $48.71
Chairs, Furniture, FUR-CH-4421
Subtotal: $48.71
Discount (20%): $9.74
Shipping: $11.13
Total: $50.10
Notes:
Thanks for your business!
Terms:
Order ID : CA-2012-AB10015140-40974


In [33]:
#Function to extract the data using line.startswith. You can also use regular expressions to match the lines you want to extract

def extract_data_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        Date, Ship_mode, Balance_due, Subtotal, Discount, Shipping, Grand_total, Order_ID = None, None, None, None, None, None, None, None
        for page in pdf.pages:
            text = page.extract_text()
            
            for idx, line in enumerate(text.split('\n')):
                if line.startswith('Date:'):
                    list = line.split()
                    Date = ' '.join(list[1:])
                if line.startswith('Ship Mode:'):
                    list = line.split()
                    Ship_mode = ' '.join(list[2:])
                if line.startswith('Balance Due:'):
                    list = line.split()
                    Balance_due = list[2]
                if line.startswith('Subtotal:'):
                    list = line.split()
                    Subtotal = list[1]
                if line.startswith('Discount'):
                    list = line.split()
                    Discount = list[2]
                if line.startswith('Shipping'):
                    list = line.split()
                    Shipping = list[1]
                if line.startswith('Total'):
                    list = line.split()
                    Grand_total = list[1]
                if line.startswith('Order ID :'):
                    list = line.split()
                    Order_ID = list[3]
                    if Order_ID != None:
                        yield(Date, Ship_mode, Balance_due, Subtotal, Discount, Shipping, Grand_total, Order_ID)

In [36]:
# The following code goes throught the data in the PDF applying the function above and appends the data to the tuple data sctructure the appends the line items
# to the dataframe called df

pdf_path = 'invoice_Aaron Bergman_36258.pdf'
for data in extract_data_from_pdf(pdf_path):
  line_items.append(data)
  df = pd.DataFrame(line_items, columns=['Date', 'Ship_mode', 'Balance_due', 'Subtotal', 'Discount', 'Shipping', 'Grand_total', 'Order_ID'])

In [8]:
# convert to excel
df.to_excel('Data.xlxs')

In [37]:
df

,Date,Ship_mode,Balance_due,Subtotal,Discount,Shipping,Grand_total,Order_ID
0,Mar 06 2012,First Class,$50.10,$48.71,$9.74,$11.13,$50.10,CA-2012-AB10015140-40974
